# Lab 15.3: Specialty Insurance Products

**Duration:** 10 minutes  
**Difficulty:** Advanced  

## Learning Objectives

In this notebook, you will:
- Create specialty insurance products (Professional Liability, Umbrella)
- Implement claims-made coverage with retroactive dates
- Build umbrella policies with underlying coverage requirements
- Analyze specialty coverage coordination

---

## Step 1: Connect to Neo4j

Establish connection and verify commercial insurance is in place.

In [ ]:
from neo4j import GraphDatabase
import pandas as pd
from datetime import datetime, date
import uuid

# Connect to Neo4j Enterprise instance
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

def run_query(query, parameters=None):
    with driver.session(database="insurance") as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]

# Verify current database state
current_state = run_query("""
MATCH (n) 
RETURN labels(n)[0] AS node_type, count(n) AS count
ORDER BY count DESC
""")

print("Current Database State:")
for record in current_state:
    print(f"  {record['node_type']}: {record['count']} nodes")

## Step 2: Create Specialty Insurance Products

### Business Context: Specialty Insurance Lines

**Professional Liability Insurance (Errors & Omissions):**
- Covers professional negligence and mistakes in services provided
- Claims-made coverage: Claim must be made during policy period
- Retroactive date: Coverage extends back to this date for prior work
- Extended Reporting Period (Tail Coverage): Option to report claims after policy ends
- Defense costs typically included within policy limits
- Common for consultants, technology firms, healthcare providers, lawyers

**Umbrella Liability Insurance:**
- Excess coverage over primary policies (auto, general liability, employers liability)
- Provides additional limits when underlying policies exhausted
- Drop-down coverage: Fills gaps not covered by underlying policies
- Required underlying limits must be maintained
- Broader coverage than underlying policies
- Defense costs typically in addition to limits
- Self-insured retention (SIR) for gaps in underlying coverage

**Key Differences from Standard Coverage:**
- Higher limits available (up to $100M for umbrella)
- More sophisticated risk assessment required
- Specialized underwriting expertise needed
- Claims handling requires coordination with underlying policies

In [ ]:
# Create specialty insurance products
specialty_products_query = """
// Create Specialty Insurance Products
CREATE (prof_liability:Product:Insurance:Specialty {
  id: randomUUID(),
  product_id: "PROD-SPEC-001",
  product_name: "Professional Liability Elite",
  product_type: "Specialty",
  insurance_line: "Professional Liability",
  coverage_type: "Claims Made",
  
  // Coverage details
  min_coverage: 1000000.00,
  max_coverage: 25000000.00,
  aggregate_coverage: "2x Per Claim",
  retroactive_date: "Available",
  
  // Professional specific
  covered_professions: ["Technology", "Consulting", "Engineering", "Healthcare"],
  regulatory_defense: true,
  disciplinary_proceedings: true,
  cyber_liability_extension: true,
  
  // Features
  worldwide_coverage: true,
  extended_reporting_period: "3 Years",
  prior_acts_coverage: true,
  
  created_at: datetime(),
  created_by: "specialty_products",
  version: 1
})

CREATE (umbrella_coverage:Product:Insurance:Specialty {
  id: randomUUID(),
  product_id: "PROD-SPEC-002",
  product_name: "Commercial Umbrella Shield",
  product_type: "Specialty",
  insurance_line: "Umbrella Liability",
  coverage_type: "Excess over Primary",
  
  // Coverage details
  min_coverage: 1000000.00,
  max_coverage: 100000000.00,
  attachment_point: "Primary Limits",
  drop_down_coverage: true,
  
  // Underlying requirements
  required_auto_liability: 1000000.00,
  required_general_liability: 1000000.00,
  required_employers_liability: 1000000.00,
  
  // Features
  worldwide_coverage: true,
  broad_form_coverage: true,
  defense_costs: "In Addition to Limits",
  
  created_at: datetime(),
  created_by: "specialty_products",
  version: 1
})

RETURN prof_liability.product_name AS professional,
       umbrella_coverage.product_name AS umbrella
"""

product_results = run_query(specialty_products_query)
print("Specialty Insurance Products Created:")
for record in product_results:
    print(f"  Professional Liability: {record['professional']}")
    print(f"  Umbrella Coverage: {record['umbrella']}")

## Step 3: Create Professional Liability Policy

### Business Context: Claims-Made Coverage

**Claims-Made vs. Occurrence:**
- **Occurrence**: Covers incidents that occur during policy period (when claim filed doesn't matter)
- **Claims-Made**: Covers claims made during policy period for covered incidents

**Retroactive Date Importance:**
- Defines earliest date for which prior acts are covered
- Usually set to policy inception date for new policies
- Should be maintained when switching carriers to avoid gaps
- Moving retroactive date forward creates gap in coverage

**Extended Reporting Period (ERP/Tail):**
- Purchased when policy canceled or not renewed
- Allows reporting claims after policy expires
- Typically 1-5 years, sometimes unlimited
- Cost usually 150-300% of final annual premium
- Essential for professionals retiring or changing carriers

In [ ]:
# Create professional liability policy
prof_liability_query = """
// Match business customer and product
MATCH (tech_corp:Customer:Business {customer_id: "CUST-BUS-001"})
MATCH (prof_product:Product {product_id: "PROD-SPEC-001"})

// Create Professional Liability Policy
CREATE (prof_policy:Policy:Specialty:Active {
  id: randomUUID(),
  policy_number: "POL-PROF-001500",
  product_type: "Specialty",
  business_type: "Professional Liability",
  
  // Coverage details
  coverage_limit: 5000000.00,
  aggregate_limit: 10000000.00,
  deductible: 25000.00,
  annual_premium: 18500.00,
  
  // Policy dates
  effective_date: date("2024-06-01"),
  expiration_date: date("2025-06-01"),
  retroactive_date: date("2016-01-01"),
  
  // Professional liability specific
  covered_services: ["Software Development", "IT Consulting", "System Integration"],
  geographic_scope: "Worldwide",
  prior_acts_coverage: true,
  regulatory_coverage: true,
  
  // Claims made provisions
  extended_reporting_period: "3 Years Available",
  discovery_period: "60 Days",
  notice_requirements: "As Soon As Practicable",
  
  // Status
  policy_status: "Active",
  premium_status: "Current",
  last_payment_date: date("2024-06-01"),
  next_payment_due: date("2024-12-01"),
  
  created_at: datetime(),
  created_by: "specialty_underwriting",
  version: 1
})

// Create Relationships
CREATE (tech_corp)-[:HOLDS_POLICY {
  start_date: prof_policy.effective_date,
  policy_role: "Named Insured"
}]->(prof_policy)

CREATE (prof_policy)-[:BASED_ON {
  underwriting_date: prof_policy.effective_date,
  risk_assessment: "Approved - Technology Risks"
}]->(prof_product)

RETURN tech_corp.business_name AS business,
       prof_policy.policy_number AS policy,
       prof_policy.coverage_limit AS coverage,
       prof_policy.retroactive_date AS retro_date,
       prof_policy.annual_premium AS premium
"""

prof_results = run_query(prof_liability_query)
print("\nProfessional Liability Policy Created:")
for record in prof_results:
    print(f"  Business: {record['business']}")
    print(f"  Policy: {record['policy']}")
    print(f"  Per Claim Limit: ${record['coverage']:,.2f}")
    print(f"  Retroactive Date: {record['retro_date']}")
    print(f"  Annual Premium: ${record['premium']:,.2f}")

## Step 4: Create Umbrella Liability Policy

### Business Context: Umbrella Coverage Structure

**How Umbrella Insurance Works:**
1. **Primary policies exhaust**: Underlying general liability pays up to $2M
2. **Umbrella attaches**: Provides additional $10M above primary limits
3. **Total protection**: $12M total coverage ($2M primary + $10M umbrella)

**Drop-Down Coverage:**
- Umbrella may cover exposures not covered by underlying policies
- Subject to self-insured retention (SIR), typically $10K-$25K
- Provides broader coverage than primary policies
- Acts as "true" excess over required underlying limits

**Underlying Policy Requirements:**
- Must maintain specified limits on primary policies
- If underlying limits reduced, umbrella may not respond
- Common requirements:
  - General Liability: $1M-$2M per occurrence
  - Auto Liability: $1M combined single limit
  - Employers Liability: $1M per accident

**Defense Costs:**
- Usually paid in addition to policy limits (unlike primary GL)
- Umbrella carrier has duty to defend when limits exceeded
- May coordinate with underlying carrier on defense

In [ ]:
# Create umbrella liability policy
umbrella_query = """
// Match business customer and product
MATCH (tech_corp:Customer:Business {customer_id: "CUST-BUS-001"})
MATCH (umbrella_product:Product {product_id: "PROD-SPEC-002"})

// Create Umbrella Liability Policy
CREATE (umbrella_policy:Policy:Specialty:Active {
  id: randomUUID(),
  policy_number: "POL-UMB-001500",
  product_type: "Specialty",
  business_type: "Umbrella Liability",
  
  // Coverage details
  coverage_limit: 10000000.00,
  deductible: 10000.00,
  annual_premium: 8750.00,
  
  // Policy dates
  effective_date: date("2024-06-01"),
  expiration_date: date("2025-06-01"),
  
  // Umbrella specific
  attachment_point: 2000000.00,
  underlying_policies: ["POL-GL-001500", "POL-WC-001500"],
  drop_down_coverage: true,
  aggregate_erosion: false,
  
  // Coverage territory
  geographic_scope: "Worldwide",
  defense_costs: "In Addition",
  broad_form_coverage: true,
  
  // Retained limits
  self_insured_retention: 10000.00,
  retention_basis: "Per Occurrence",
  
  // Status
  policy_status: "Active",
  premium_status: "Current",
  last_payment_date: date("2024-06-01"),
  next_payment_due: date("2024-12-01"),
  
  created_at: datetime(),
  created_by: "specialty_underwriting",
  version: 1
})

// Create Relationships
CREATE (tech_corp)-[:HOLDS_POLICY {
  start_date: umbrella_policy.effective_date,
  policy_role: "Named Insured"
}]->(umbrella_policy)

CREATE (umbrella_policy)-[:BASED_ON {
  underwriting_date: umbrella_policy.effective_date,
  risk_assessment: "Approved - Excess Coverage"
}]->(umbrella_product)

RETURN tech_corp.business_name AS business,
       umbrella_policy.policy_number AS policy,
       umbrella_policy.coverage_limit AS coverage,
       umbrella_policy.attachment_point AS attachment,
       umbrella_policy.annual_premium AS premium
"""

umbrella_results = run_query(umbrella_query)
print("\nUmbrella Liability Policy Created:")
for record in umbrella_results:
    print(f"  Business: {record['business']}")
    print(f"  Policy: {record['policy']}")
    print(f"  Umbrella Limit: ${record['coverage']:,.2f}")
    print(f"  Attachment Point: ${record['attachment']:,.2f}")
    print(f"  Annual Premium: ${record['premium']:,.2f}")

## Step 5: Specialty Coverage Analysis Queries

Analyze specialty coverage coordination and limits stacking.

In [ ]:
# Query 1: Complete coverage tower analysis
coverage_tower_query = """
MATCH (b:Customer:Business {customer_id: "CUST-BUS-001"})-[:HOLDS_POLICY]->(p:Policy)
WHERE p.policy_status = "Active"
WITH b,
     collect({
       policy_type: p.business_type,
       policy_number: p.policy_number,
       limit: p.coverage_limit,
       premium: p.annual_premium,
       attachment: COALESCE(p.attachment_point, 0)
     }) AS policies,
     sum(p.coverage_limit) AS total_limits,
     sum(p.annual_premium) AS total_premium
RETURN b.business_name AS business,
       size(policies) AS active_policies,
       total_limits,
       total_premium,
       policies
"""

tower_results = run_query(coverage_tower_query)
print("\nCoverage Tower Analysis:")
for record in tower_results:
    print(f"  Business: {record['business']}")
    print(f"  Active Policies: {record['active_policies']}")
    print(f"  Total Coverage Limits: ${record['total_limits']:,.2f}")
    print(f"  Total Annual Premium: ${record['total_premium']:,.2f}")
    print("\n  Coverage Layers:")
    for policy in sorted(record['policies'], key=lambda x: x['attachment']):
        if policy['attachment'] > 0:
            print(f"    {policy['policy_type']}: ${policy['limit']:,.2f} xs ${policy['attachment']:,.2f}")
        else:
            print(f"    {policy['policy_type']}: ${policy['limit']:,.2f} primary")

In [ ]:
# Query 2: Specialty policy features comparison
specialty_features_query = """
MATCH (p:Policy:Specialty)-[:BASED_ON]->(prod:Product:Specialty)
MATCH (b:Customer)-[:HOLDS_POLICY]->(p)
RETURN b.business_name AS business,
       p.policy_number AS policy,
       prod.insurance_line AS line,
       prod.coverage_type AS coverage_type,
       p.coverage_limit AS limit,
       p.geographic_scope AS territory,
       COALESCE(p.retroactive_date, 'N/A') AS retro_date,
       p.annual_premium AS premium
ORDER BY p.annual_premium DESC
"""

specialty_df = pd.DataFrame(run_query(specialty_features_query))
print("\nSpecialty Policy Features:")
print(specialty_df.to_string(index=False))

In [ ]:
# Query 3: Underlying policy verification for umbrella
underlying_verification_query = """
MATCH (umb:Policy {business_type: "Umbrella Liability"})
MATCH (b:Customer)-[:HOLDS_POLICY]->(umb)
MATCH (b)-[:HOLDS_POLICY]->(underlying:Policy)
WHERE underlying.policy_number IN umb.underlying_policies
RETURN b.business_name AS business,
       umb.policy_number AS umbrella_policy,
       umb.attachment_point AS umbrella_attachment,
       underlying.policy_number AS underlying_policy,
       underlying.business_type AS underlying_type,
       underlying.coverage_limit AS underlying_limit,
       CASE 
         WHEN underlying.coverage_limit >= umb.attachment_point 
         THEN 'Adequate' 
         ELSE 'Gap in Coverage' 
       END AS limit_adequacy
"""

underlying_df = pd.DataFrame(run_query(underlying_verification_query))
print("\nUnderlying Policy Verification:")
print(underlying_df.to_string(index=False))

In [ ]:
# Query 4: Specialty product performance metrics
specialty_performance_query = """
MATCH (prod:Product:Specialty)<-[:BASED_ON]-(p:Policy:Specialty)
WITH prod,
     count(p) AS policies_written,
     sum(p.annual_premium) AS total_premium,
     avg(p.annual_premium) AS avg_premium,
     avg(p.coverage_limit) AS avg_limit,
     sum(p.coverage_limit) AS total_limits
RETURN prod.product_name AS product,
       prod.insurance_line AS line,
       prod.coverage_type AS type,
       policies_written,
       round(total_premium, 2) AS total_premium,
       round(avg_premium, 2) AS avg_premium,
       round(avg_limit, 2) AS avg_limit
ORDER BY total_premium DESC
"""

performance_df = pd.DataFrame(run_query(specialty_performance_query))
print("\nSpecialty Product Performance:")
print(performance_df.to_string(index=False))

## Step 6: Summary

### What You've Accomplished:

- Created **Professional Liability (E&O)** products with claims-made coverage
- Created **Umbrella Liability** products providing excess coverage over primary policies
- Implemented retroactive dates for professional liability coverage
- Built coverage towers with coordinated limits and attachment points
- Established underlying policy requirements for umbrella coverage
- Analyzed specialty coverage coordination and limit adequacy

### Key Specialty Insurance Concepts:

1. **Claims-Made Coverage**: Understanding time-based coverage triggers and retroactive dates
2. **Coverage Towers**: How excess policies layer over primary coverage to create protection
3. **Attachment Points**: The point at which umbrella coverage begins to respond
4. **Drop-Down Coverage**: Umbrella filling gaps in underlying coverage subject to SIR
5. **Extended Reporting Periods**: Tail coverage for claims-made policies when coverage ends
6. **Defense Costs**: In addition to vs. within policy limits significantly impacts effective coverage

### Coverage Tower Example:

For the technology company:
- **Primary General Liability**: $2M per occurrence (ground layer)
- **Umbrella Liability**: $10M excess of $2M (second layer)
- **Total Protection**: $12M total liability coverage
- **Professional Liability**: $5M per claim (separate tower for professional risks)

---

**Next:** Continue to notebook 04 for Reinsurance Networks.